In [ ]:
# Cell 1: Install Dependencies
# This cell installs all the required libraries with their specific versions.
%pip install tensorflow==2.16.1
%pip install numpy==1.26.4
%pip install pandas==2.2.2
%pip install scikit-learn==1.5.0
%pip install tensorflow-datasets==4.9.4
%pip install mlflow==2.14.1
%pip install matplotlib==3.9.0
%pip install seaborn==0.13.2



In [ ]:
# Cell 2: Main Experiment Script
import mlflow
import tensorflow as tf
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- Keras Callback for MLflow Logging ---
class MLflowLogger(tf.keras.callbacks.Callback):
    """Keras callback to log metrics for each epoch."""
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        mlflow.log_metric('epoch_loss', logs.get('loss'), step=epoch)
        mlflow.log_metric('epoch_accuracy', logs.get('accuracy'), step=epoch)
        mlflow.log_metric('epoch_val_loss', logs.get('val_loss'), step=epoch)
        mlflow.log_metric('epoch_val_accuracy', logs.get('val_accuracy'), step=epoch)

def plot_confusion_matrix(y_true, y_pred, class_names):
    """Generates and saves a confusion matrix plot."""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    
    # In Kaggle, files are often saved in /kaggle/working/
    if not os.path.exists("artifacts"):
        os.makedirs("artifacts")
        
    path = os.path.join("artifacts", "confusion_matrix.png")
    plt.savefig(path)
    plt.close()
    return path

def run_experiment():
    # --- 1. Load and Preprocess Data ---
    print("Fetching and preparing MNIST dataset...")
    mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)
    X, y = mnist["data"], mnist["target"].astype(np.uint8)

    # Normalize pixel values to be between 0 and 1
    X = X / 255.0
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Reshape data for the neural network
    X_train = X_train.reshape(-1, 28, 28)
    X_test = X_test.reshape(-1, 28, 28)
    
    class_names = np.unique(y).astype(str)
    print("Dataset ready.")

    # --- 2. Define Hyperparameters and Model ---
    params = {
        'epochs': 10,
        'batch_size': 64,
        'learning_rate': 0.001
    }
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # --- 3. MLflow Experiment Tracking ---
    mlflow.set_experiment("Advanced MNIST Classification (Neural Network)")

    with mlflow.start_run() as run:
        print(f"Starting MLflow Run: {run.info.run_name}")
        mlflow.log_params(params)

        print("Training model...")
        model.fit(X_train, y_train, 
                  validation_split=0.2,
                  epochs=params['epochs'], 
                  batch_size=params['batch_size'],
                  callbacks=[MLflowLogger()],
                  verbose=2)
        print("Model training completed.")
        
        # --- 4. Log Artifacts ---
        # Log the final model
        mlflow.keras.log_model(model, artifact_path="model")

        # Generate predictions for evaluation
        y_pred_probs = model.predict(X_test)
        y_pred = np.argmax(y_pred_probs, axis=1)

        # Generate and log the confusion matrix
        print("Generating and logging confusion matrix...")
        cm_path = plot_confusion_matrix(y_test, y_pred, class_names)
        mlflow.log_artifact(cm_path, "plots")
        
    print("\n--- MLflow Run Completed ---")
    print("You can now view the results in the MLflow UI.")

# Run the main function
run_experiment()



# Cell 3: Viewing the Results

### How to View the MLflow UI

After running the experiment in the cell above, a new directory named `mlruns` will be created in your notebook's environment. This directory contains all the tracking data.

To view the interactive MLflow dashboard, you would typically run the command `mlflow ui` from a terminal in the same directory. However, in environments like Kaggle or Colab, you can't directly access the local server.

**Option 1: Download the `mlruns` directory**

You can download the entire `mlruns` folder, run `mlflow ui` on your local machine from the directory where you downloaded it, and explore the results there.

**Option 2: Using `ngrok` (Advanced)**

For a live view, you can use a tool like `ngrok` to create a public URL for the MLflow UI server. This involves running the UI in the background and then exposing its port (usually 5000) through ngrok.

Example commands:
```bash
# Install ngrok
!pip install pyngrok

# Run mlflow ui in the background
!mlflow ui --host 0.0.0.0 &

# Expose the port using ngrok
from pyngrok import ngrok
public_url = ngrok.connect(5000)
print(f"Click this link to view the MLflow UI: {public_url}")
```
You would run these in a new code cell after the experiment is complete.



In [ ]:
# Cell 4: Package and Download Results
import shutil
import os

# The MLflow data is stored in the 'mlruns' directory.
# We will compress this directory into a zip file for easy download.
if os.path.exists('mlruns'):
    shutil.make_archive('mlruns_results', 'zip', 'mlruns')
    print("Successfully created 'mlruns_results.zip'.")
    print("\n--> You can now find 'mlruns_results.zip' in the file viewer on the right-hand side of your Kaggle notebook.")
    print("--> Click on it and then select 'Download' to save it to your local machine.")
else:
    print("The 'mlruns' directory was not found. Please run the experiment cell (Cell 2) first.")

